# Data collection

As my goal is to learn data science, I want to use only data provided directly by Riot Games, the developers of League of Legends. I will utilize their API to extract data, ensuring that the data is as raw and unprocessed as possible.

Note: Riot's API key for amateur developers is only valid for 24 hours after generation. Please remember to refresh your key regularly and store it in the `.env` file to ensure the project's code functions correctly.